In [109]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
from lib.reproduction import major_oxides
from sklearn.metrics import mean_squared_error
from lib import full_flow_dataloader
import mlflow
import numpy as np
import datetime
import os

os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras
import mlflow.keras


torch.manual_seed(42)
np.random.seed(42)

In [111]:
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [112]:
train_processed, test_processed = full_flow_dataloader.load_full_flow_data(load_cache_if_exits=True, average_shots=True)

In [117]:
import pandas as pd 
ica_X_train = pd.read_csv("/home/christian/projects/p9/baseline/data/_preformatted_ica/norm1/ica_data_edbf0b20b6488320823dbfdd75835e58.csv")



In [119]:
ICA_X_train_reshaped = ica_X_train.drop(columns=['Sample Name', 'ID']).to_numpy().reshape(-1, 6144, 1)

In [113]:
from keras import layers, optimizers, regularizers


def build_model(input_dim, output_dim):
    model = keras.models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Reshape((48, 128, 1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Additional convolutional block for better feature extraction
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(output_dim))
    
    # Using L2 regularization
    model.add(layers.Dense(output_dim, kernel_regularizer=regularizers.l2(0.01)))
    
    # Optimizer with a custom learning rate
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['root_mean_squared_error', 'mae'])
    return model


# Constants
INPUT_DIM = 6144  # Number of features per sample
OUTPUT_DIM = 8  # Number of continuous values as output

# Model Creation
model = build_model(INPUT_DIM, OUTPUT_DIM)

model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_24 (Reshape)            │ (None, 2048, 3, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 2048, 3, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 2048, 3, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 1024, 1, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 1024, 1, 32)    │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 1024, 1, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 1024, 1, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 1024, 1, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_60 (Conv2D)              │ (None, 1024, 1, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 1024, 1, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 256)            │    33,554,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,659,688 (128.40 MB)

 Trainable params: 33,659,176 (128.40 MB)

 Non-trainable params: 512 (2.00 KB)

In [114]:
drop_cols = major_oxides + ["ID", "Sample Name"]

X_train = train_processed.drop(columns=drop_cols)
y_train = train_processed[major_oxides]

X_test = test_processed.drop(columns=drop_cols)
y_test = test_processed[major_oxides]


X_train_reshaped = X_train.to_numpy().reshape(-1, 6144, 1)
X_test_reshaped = X_test.to_numpy().reshape(-1, 6144, 1)

In [115]:
callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)


def run_experiment(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: keras.Model,
    epochs: int,
    batch_size: int,
    callbacks: list = [],
    major_oxides: list = [],
):
    with mlflow.start_run(run_name="CNN"):
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.1,
            callbacks=callbacks,
        )
        y_pred = model.predict(X_test)
        for i, oxide in enumerate(major_oxides):
            y_test_oxide = y_test[:, i]
            y_pred_oxide = y_pred[:, i]
            rmse = np.sqrt(mean_squared_error(y_test_oxide, y_pred_oxide))
            mlflow.log_metric(f"rmse_{oxide}", float(rmse))

In [116]:
mlflow.keras.autolog()
mlflow.set_experiment(f'3_CNN_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')

run_experiment(
    X_train_reshaped,
    y_train.to_numpy(),
    X_test_reshaped,
    y_test.to_numpy(),
    model,
    epochs=1000,
    batch_size=256,
    callbacks=[callback],
    major_oxides=major_oxides,
)

2024/04/18 15:56:30 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of keras. If you encounter errors during autologging, try upgrading / downgrading keras to a supported version, or try upgrading MLflow.
2024/04/18 15:56:34 INFO mlflow.tracking.fluent: Experiment with name '3_CNN_20240418-155630' does not exist. Creating a new experiment.


Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 7224.1855 - mae: 40.7675 - root_mean_squared_error: 80.3436 - val_loss: 488.7612 - val_mae: 11.1966 - val_root_mean_squared_error: 22.1045
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 219ms/step - loss: 450.3408 - mae: 13.5791 - root_mean_squared_error: 21.2149 - val_loss: 504.2368 - val_mae: 11.4398 - val_root_mean_squared_error: 22.4518
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 229ms/step - loss: 419.9860 - mae: 11.4264 - root_mean_squared_error: 20.4895 - val_loss: 496.4176 - val_mae: 11.2943 - val_root_mean_squared_error: 22.2771
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 537ms/step - loss: 396.5294 - mae: 12.7709 - root_mean_squared_error: 19.9090 - val_loss: 487.8821 - val_mae: 11.1805 - val_root_mean_squared_error: 22.0847
Epoch 5/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 509ms/step - loss: 373.1249 - mae: 11.8938 - root_mean_squared_error: 19.3113 - val_loss: 472.4716 - val_mae: 11.0015 - val_root_mean_squared_error: 21.7330
Epoch 6/100